In [ ]:
from tifffile import imread, imwrite
from glob import glob
import os
from efficient_v2_unet.model.predict import predict

# Predict images and save them

This notebook shows how to use a EfficientV2-UNet model to predict images.

Images will be tiled (with overlap) for prediction and merged back together for the final output.


## FIXME maybe use skimage.io.imsave

In [ ]:
# Variables

image_folder_path = 'path/to/image/folder'
output_path = 'path/to/desired/output/folder'
model_path = 'path/to/model/model.h5'
image_file_extension = '.tif'
my_threshold = 0.5

In [ ]:
# Load images

image_paths = glob(image_folder_path + '/*' + image_file_extension)
images = [imread(x) for x in image_paths]

# Predict images
predicted_images = predict(
    images=images,
    model=model_path,
    threshold=my_threshold,      # default is None (i.e. no thresholding)
    factor=1,           # default
    batch_size=0        # default is 0 (=>32)
)

# Save images
for img, path in zip(predicted_images, image_paths):
    out_image_name = os.path.basename(path)
    out_image_path = os.path.join(output_path, out_image_name)
    imwrite(out_image_path, img)
    print('Saved image to:', out_image_path)

### Parameters explained

**images**: single numpy array or a list of numpy arrays

**model**: can be a string path to the model.h5 file or an already loaded model.

**threshold**: the threshold to be used on the prediction map. If `None`, it will save the prediction maps.

**factor**: the downscaling factor to predict the images on. E.g. `factor=2` will half the resolution (and image size) just before prediction.
After the prediction the image is rescaled to the original image size.

**batch_size**: number of image-tiles to be predicted in a batch. The default is 0, which sets it to 32. When memory is problem, the value can be decreased by factors of 2 (e.g. 16, ..., 2, 1)

### Other parameters

**tile_size**: default is 512. Size of tiling in pixels (e.g. 512x512)

**overlap**: overlap between tiles in pixels, default is 0, which takes 15% of the tile size.